In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.misc 
import scipy.io


import urllib.request
import shutil
from os import makedirs, listdir
from os.path import join, isfile, isdir, exists, splitext

cwd = os.getcwd()
VGG_PATH = cwd + "/Weights/imagenet-vgg-verydeep-19.mat"

if not isfile(VGG_PATH):
    url = 'http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat'
    with urllib.request.urlopen(url) as res, open(VGG_PATH, 'wb') as f:
        print ("Download . . . ")
        shutil.copyfileobj(res, f)

print ("Packages loaded.")

Download . . . 
Packages loaded.


In [2]:
def net(data_path, input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    data = scipy.io.loadmat(data_path)
    mean = data['normalization'][0][0][0]
    mean_pixel = np.mean(mean, axis=(0, 1))
    weights = data['layers'][0]

    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if name == 'conv1_1':
            kernels, bias = weights[i][0][0][0][0]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            
            current = _conv_layer(current, kernels, bias)
        elif kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current)
        net[name] = current
        print(name)
        
    assert len(net) == len(layers)
    
    
    return net, mean_pixel

def _conv_layer(input, weights, bias):
    print (weights.shape)
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)
def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')
def preprocess(image, mean_pixel):
    return image - mean_pixel
def unprocess(image, mean_pixel):
    return image + mean_pixel

In [3]:


with tf.Graph().as_default(), tf.Session() as sess:
    with tf.device("/cpu:0"):
        img_placeholder = tf.placeholder(tf.float32, shape=(None, 112, 112, 3))
        net, mean_pixel = net(VGG_PATH, img_placeholder)
        
        
print("Done")


(3, 3, 3, 64)
conv1_1
relu1_1
(3, 3, 64, 64)
conv1_2
relu1_2
pool1
(3, 3, 64, 128)
conv2_1
relu2_1
(3, 3, 128, 128)
conv2_2
relu2_2
pool2
(3, 3, 128, 256)
conv3_1
relu3_1
(3, 3, 256, 256)
conv3_2
relu3_2
(3, 3, 256, 256)
conv3_3
relu3_3
(3, 3, 256, 256)
conv3_4
relu3_4
pool3
(3, 3, 256, 512)
conv4_1
relu4_1
(3, 3, 512, 512)
conv4_2
relu4_2
(3, 3, 512, 512)
conv4_3
relu4_3
(3, 3, 512, 512)
conv4_4
relu4_4
pool4
(3, 3, 512, 512)
conv5_1
relu5_1
(3, 3, 512, 512)
conv5_2
relu5_2
(3, 3, 512, 512)
conv5_3
relu5_3
(3, 3, 512, 512)
conv5_4
relu5_4
Done
